In [ ]:
# Import random graphs.
from networkx.generators.random_graphs import erdos_renyi_graph
from networkx.generators.random_graphs import barabasi_albert_graph
from networkx.generators.community import stochastic_block_model
from networkx.generators.random_graphs import watts_strogatz_graph
from networkx.generators.community import random_partition_graph

# Standard imports
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

#import math
from tqdm import tqdm
#import seaborn as sns
#from sklearn.decomposition import FactorAnalysis

#import random
#from deeprobust.graph.data import Dataset

The code in this notebook is adapted from the paper "A Unified Framework for Optimization-Based
Graph Coarsening\" by Kumar et. al (2023).

# Graph Coarsening Algorithm

In [ ]:
class GraphCoarsener:
    def __init__(self, L, X, n, gamma, lambd, alpha, seed = 666):
        # Save parameter fields.
        self.L = L
        self.X = X
        self.N = X.shape[0]
        self.n = n
        self.d = X.shape[1]
        
        # Initialize optimization variables as random matrices.
        np.random.seed(seed)
        self.X_tilde = np.random.normal(0, 1, (n, d))
        
        self.clip = 1e-10
        self.C = np.random.normal(0, 1, (N, n))
        self.C[self.C < self.clip] = self.clip

        # Save regularization variables.
        self.alpha = alpha
        self.lambd = lambd
        self.gamma = gamma
        
        # Optimization parameters.
        self.num_iter = 0
        self.lr = 1e-5
        NUM_C_ITER = 100

    def objective(self):
        f = 0
        
        # Bregman divergence.
        J = np.outer(np.ones(self.n), np.ones(self.n)) / self.n
        f += -self.gamma * np.linalg.slogdet(self.C.T @ self.L @ self.C + J)[1]
        
        # Dirichlet energy.
        L_tilde = self.C.T @ self.L @ self.C
        f += np.trace(self.X_tilde.T @ L_tilde @ self.X_tilde)

        # Regularization for C.
        f += self.lambd * np.linalg.norm(self.C @ np.ones((self.n, 1)))**2 / 2
        
        # Regularization for X_tilde.
        f += self.alpha * np.linalg.norm(self.X - self.C @ self.X_tilde)**2 / 2
        
        return f

    def update_X_tilde(self):
        L_tilde = self.C.T @ self.L @ self.C
        A = 2 * L_tilde / self.alpha + self.C.T @ self.C
        self.X_tilde = np.linalg.pinv(A) @ self.C.T @ X

        for i in range(len(self.X_tilde)):
            self.X_tilde[i] = self.X_tilde[i] / np.linalg.norm(self.X_tilde[i])
        
        return None

    def gradient_C(self):
        grad = np.zeros(self.C.shape)
        
        J = np.outer(np.ones(self.n), np.ones(self.n)) / self.n
        v = np.linalg.pinv(self.C.T @ self.L @ self.C + J)
        grad += -2*self.gamma * self.L @ self.C @ v
        
        grad += self.alpha * (self.C @ self.X_tilde - self.X) @ self.X_tilde.T
        grad += 2*self.L @ self.C @ self.X_tilde @ self.X_tilde.T
        grad += self.lambd * np.abs(self.C) @ (np.ones((self.n, self.n)))

        return grad

    def update_C(self):
        self.C = self.C - self.lr * self.gradient_C()
        self.C[self.C < self.clip] = self.clip

        for i in range(len(self.C)):
            self.C[i] = self.C[i] / np.linalg.norm(self.C[i],1)
        
        return None

    def fit(self, num_iters):
        loss = np.zeros(num_iters)
        for i in tqdm(range(num_iters)):
            for _ in range(NUM_C_ITER):
                self.update_C()
            self.update_X_tilde()
            loss[i] = self.objective()
            self.num_iter += 1
        return (self.C, self.X_tilde, loss)

# Generate Random Data

In [10]:
# Some Input parameters.
np.random.seed(666)
N = 500 # number of nodes
p = 0.1
#graph = erdos_renyi_graph(N, p, directed = False)
#graph = nx.barabasi_albert_graph(n = N, m = 20)
#graph = watts_strogatz_graph(N, 20, p, seed = 666)
graph = nx.random_geometric_graph(N, p)


# DISPLAY GENERATED GRAPH.
#print(graph.edges)
#print(graph.nodes)

# PLOTTING GENERATED GRAPH.
#nx.draw(graph, with_labels = True)
#plt.title("Laplacian")
#plt.show()

# CREATING EDGE WEIGHTS.
np.random.seed(666)
W = np.zeros((N, N))
for (x, y) in graph.edges:
    W[x][y] = np.random.randint(1,10)
W = W + W.T

# CALCULATING LAPLACIAN MATRIX OF GENERATED GRAPH.
D = np.diag(W @ np.ones((W.shape[0])))
L = D - W
print(L)
print(L.shape)

[[ 65.   0.   0. ...   0.   0.   0.]
 [  0.  90.  -9. ...   0.   0.   0.]
 [  0.  -9. 145. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...  96.   0.   0.]
 [  0.   0.   0. ...   0.  97.   0.]
 [  0.   0.   0. ...   0.   0.  27.]]
(500, 500)


In [11]:
# Create features for the synthetic graph
d = 250
X = np.random.multivariate_normal(np.zeros(N), np.linalg.pinv(L), d).T
X.shape
print(X)

[[ 3.25374963e-02  1.79055906e-01  2.13773962e-02 ...  1.89392113e-01
  -1.47952522e-01 -2.29413576e-01]
 [ 1.13106317e-01 -2.25278824e-01  1.75945720e-02 ... -3.16192104e-01
  -4.84635585e-02 -1.09480817e-01]
 [ 2.94350902e-01 -1.69274694e-01 -6.18232059e-02 ... -1.91962654e-01
   1.41152207e-02  2.26638180e-01]
 ...
 [-1.24959422e-01 -2.66503152e-01  7.34824476e-02 ... -2.78030129e-01
  -3.63077697e-02 -3.70847386e-02]
 [ 2.54938151e-04  1.81113082e-01 -1.39548050e-01 ... -4.98979170e-02
  -1.14394233e-02  7.64733940e-02]
 [ 1.52676741e-01  2.42150694e-01  3.23388109e-01 ... -2.07616538e-01
   2.58863465e-01  4.00311933e-01]]


In [22]:
class GraphCoarsener:
    def __init__(self, L, X, n, gamma, lambd, alpha, seed = 666):
        # Save parameter fields
        self.L = L
        self.X = X
        self.N = X.shape[0]
        self.n = n
        self.d = X.shape[1]
        
        # Randomly initialize optimization variables
        np.random.seed(seed)
        self.X_tilde = np.random.normal(0, 1, (n, d))
        
        self.clip = 1e-10
        self.C = np.random.normal(0, 1, (N, n))
        self.C[self.C < self.clip] = self.clip

        # Save regularization variables
        self.alpha = alpha
        self.lambd = lambd
        self.gamma = gamma
        
        # Optimization parameters
        self.num_iter = 0
        self.lr = 1e-5
        MAX_C_ITER = 100

    def objective(self):
        f = 0
        
        # Bregman divergence
        J = np.outer(np.ones(self.n), np.ones(self.n)) / self.n
        f += -self.gamma * np.linalg.slogdet(self.C.T @ self.L @ self.C + J)[1]
        
        # Dirichlet energy
        L_tilde = self.C.T @ self.L @ self.C
        f += np.trace(self.X_tilde.T @ L_tilde @ self.X_tilde)
        
        # Regularization for X_tilde
        f += self.alpha * np.linalg.norm(self.X - self.C @ self.X_tilde)**2 / 2
        
        # Regularization for C
        f += self.lambd * np.linalg.norm(self.C @ np.ones((self.n, 1)))**2 / 2
        
        return f

    def update_X_tilde(self):
        L_tilde = self.C.T @ self.L @ self.C
        A = 2 * L_tilde / self.alpha + self.C.T @ self.C
        self.X_tilde = np.linalg.pinv(A) @ self.C.T @ X

        for i in range(len(self.X_tilde)):
            self.X_tilde[i] = self.X_tilde[i] / np.linalg.norm(self.X_tilde[i])
        
        return None

    def gradient_C(self):
        grad = np.zeros(self.C.shape)
        
        J = np.outer(np.ones(self.n), np.ones(self.n)) / self.n
        v = np.linalg.pinv(self.C.T @ self.L @ self.C + J)
        grad += -2*self.gamma * self.L @ self.C @ v
        
        grad += self.alpha * (self.C @ self.X_tilde - self.X) @ self.X_tilde.T
        grad += 2*self.L @ self.C @ self.X_tilde @ self.X_tilde.T
        grad += self.lambd * np.abs(self.C) @ (np.ones((self.n, self.n)))

        return grad

    def update_C(self):
        self.C = self.C - self.lr * self.gradient_C()
        self.C[self.C < self.clip] = self.clip

        for i in range(len(self.C)):
            self.C[i] = self.C[i] / np.linalg.norm(self.C[i],1)
        
        return None

  
    def fit(self, max_iters):
        loss = np.zeros(max_iters)
        for i in tqdm(range(max_iters)):
            for _ in range(MAX_C_ITER):
                self.update_C()
            self.update_X_tilde()
            loss[i] = self.objective()
            self.num_iter += 1
        return (self.C, self.X_tilde, loss)

In [26]:
n = 100
num_iter = 10

# Hyperparameters: lambda, alpha, gamma
obj = solver(L, X, n, 500, 500, X.shape[1]/2) 
C_0, X_t_0, loss_ls = obj.fit(num_iter)

100%|██████████████████████████████████████████| 10/10 [00:18<00:00,  1.88s/it]


In [27]:
loss_ls

[1042858.5469177227,
 927114.6925468271,
 885678.6492330466,
 868867.2941789497,
 859007.1543449631,
 854620.1061137915,
 852924.1605778319,
 851692.4810978615,
 850895.1787722268,
 850370.4848242351]

In [28]:
obj_2 = GraphCoarsener(L, X, n, X.shape[1]/2, 500, 500)
C_2, X_2, loss_2 = obj_2.fit(iterations)

100%|██████████████████████████████████████████| 10/10 [00:19<00:00,  1.92s/it]


In [29]:
loss_2

array([1081968.33008207,  940380.21633698,  868796.89777601,
        851307.8478082 ,  846549.27728529,  844743.60925996,
        843654.88322882,  842966.21609628,  842469.11532114,
        842082.34809686])